In [1]:
import cv2 
import matplotlib.pyplot as plt
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
import time  

In [4]:

class HandLandmark:
    MODEL_ASSET_PATH = 'Google Landmark models\\hand_landmarker.task'
    HANDS = 2
    Min_HDC = 0.5
    Min_HPC = 0.5
    TASK = 'image'
    base_options = python.BaseOptions(model_asset_path=MODEL_ASSET_PATH)
    options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=HANDS, min_hand_detection_confidence = Min_HDC, min_hand_presence_confidence = Min_HPC, min_tracking_confidence = 0.9)
    DETECTOR = vision.HandLandmarker.create_from_options(options)
    
    def annotate_image(self, image, detection_result):
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
        for landmarks in detection_result.hand_landmarks:
            # Initialize variables to find the bounding box
            x_min = float('inf')
            x_max = float('-inf')
            y_min = float('inf')
            y_max = float('-inf')
            for landmark in landmarks:
                x = int(landmark.x * image.shape[1])
                y = int(landmark.y * image.shape[0])
                x_min = min(x_min, x)
                x_max = max(x_max, x)
                y_min = min(y_min, y)
                y_max = max(y_max, y)
                cv2.circle(image_rgb, (x, y), 5, (0, 255, 0), -1)
            cv2.rectangle(image_rgb, (x_min, y_min), (x_max, y_max), (255, 0, 0), 1)    
        return image_rgb

    def get_landmarks(self, image):
        detection_result = self.DETECTOR.detect(image)
        return detection_result

    def draw_image_landmarks(self, image): 
        image = cv2.resize(image, (image.shape[1]//2, image.shape[0]//2))
        image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
        detection_result = self.get_landmarks(image)
        annotated_image = self.annotate_image(image.numpy_view(), detection_result)
        cv2.imshow("Hand Landmarks", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
        if self.TASK == 'image':
            cv2.waitKey(3000) 
            cv2.destroyAllWindows()

detector = HandLandmark()
detector.TASK = 'video'

cp = cv2.VideoCapture(0)
while True:
    success, image = cp.read()
    if not success:
        break
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    detector.draw_image_landmarks(image)
cv2.destroyAllWindows()
    

C:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
